<a id="Top"></a>
# Generate a Subset Selector for a Large Dataset

## Table of Contents
- [Problem Statement](#Problem)
- [Solution Principles](#Solution)
- [Original Use Case and Practical Approach](#UseCase)
    - [A Note about Churn Rate and Class Balance](#Balance)
- [Import Necessary Libraries & Packages](#Imports)
- [Set the parameters for building the ID subset](#Parameters)
- [Set the browser client timezone](#Timezone)
- [Define a function for generating filenames](#Filenames)
- [Provide the input schema if necessary](#Schema)
- [Read and merge the dataset](#Merge)
- [Sample the full list of IDs for the desired total number and class balance](#Sample)
- [Write the selected IDs to disk](#Write)

<a id="Problem"></a>
## Problem Statement
Sometimes, the data provided by a client is too voluminous to deal with productively. While in general more data is better for training ML models, there may be diminishing returns beyond a certain size. More important, in the early stages of data science exploration, it is important to experiment and iterate rapidly on model types, hyperparameter grids, feature sets, and more; this work is significantly impeded
if every run takes more than a few minutes.

We may assume that we can rely on the platform to come to the rescue: if we run Spark and give it enough nodes to run on, any size dataset can be tamed and dealt with in a reasonable amount of time by distributing the computational load. But this only goes so far. First, we usually do not have unlimited hardware or virtual resources. Second, distributed computing comes with overhead, which limits the gains that can be achieved. Finally, sheer size still makes large datasets difficult to handle; just the file or database I/O can add a lot to the time things take.

[Top **⤒**](#Top)

<a id="Solution"></a>
## Solution Principles
Given the assumption that model performance may not improve much with more data beyond a certain dataset size, and the desire to keep the dataset small enough to be manageable during iterative data science work, working with a subset instead of the full dataset is an obvious solution. The question is how to construct the subset.

There are a few considerations.
- The subset must be representative of the full dataset, i.e., selection must not be obviously biased in time or other factors.
- The use case is a binary classification model, which allows us to talk about the class balance of the dataset.
- The subset can have a different class balance from the full dataset. In fact, subsetting can help address unbalanced cases by preferentially selecting from the smaller class. This can avoid the need for other ways of addressing the class imbalance, such as data duplication and synthetic data generation.
- The subset can be based on random sampling, but must be repeatable, for consistency in comparing different models and techniques, and variable, to have the ability to generate multiple subsets and test for model bias.

[Top **⤒**](#Top)

<a id="UseCase"></a>
## Original Use Case and Practical Approach
This notebook is based on a *customer churn* use case. The data was voluminous in part because it was not aggregated; instead, the tables provided contained one row per day per customer, over a period of ten months. Over the whole period, there were almost nine million unique customer IDs; not all of them were present for the whole ten months (due to customers leaving—churning—or joining along the way). Multiple tables were provided in files representing a single month, each file containing 30 or 31 (or 28) records per customer.

The goal of this notebook was to extract a list of unique customer IDs that formed a subset of a given size, with a specified class balance expressed as the percentage of churners. As a refinement, the ability to ensure that every selected customer ID has at least $N$ months of history in the dataset was added.

In this generalized notebook, *churn* is represented as the *positive class* and *not churn* as the *negative class*. The positive class is often the smaller one (as in anomaly detection), but this is not a requirement for this notebook to work. The unique identifier (customer or contract ID in the original use case) is assumed to be a single column, referred to simply as the ID column.

The actual approach taken has the following elements.
- Use Spark to take advantage of more distributed computing (or, at least, multiple worker threads) for data manipulation operations than can be provided by straight Python with Pandas.
- Read data for a single table (schema) from multiple files. Assume that the filenames can be constructed in a consistent manner, so this can be encapsulated in a customizable function.
  - Here, "multiple" files could mean just one; in that case, there is an unnecessary level of indirection (you could just read the data once and be done with it) but it doesn't hurt.
  - Assume that the file type is CSV (compressed or uncompressed). Other file types require changes to the sample code.
- Accumulate the entire list of unique identifying values (a parameter identifies the column name) and maintain the associated labels (again, from a parameterized column). Drop all other columns.
  - Because a given identifier can appear many times in the input data, across files and within any file, assigning a single label (positive or negative, i.e., 0 or 1) involves some kind of aggregation. For a churn use case this would just be the `max()` function: 0 (not-churn) if all labels for this ID are 0; otherwise (if there are any 1s) 1. Depending on the use case, a different aggregate may be required.
- Given that each of the files may be large and take a long time to process, report progress after each file instead of waiting till the end.
- Consider only those IDs that meet the criterion of being present for a minimum number of periods. A customer who leaves in the first month of data collection will not exhibit any pattern that predictions may be based on, nor would a customer who joins in the last period of data collection.
- Use an available pyspark DataFrame method for subsampling with specified class fractions. This method uses seeded randomization, which supports repeatability (using the same seed) as well as variation (using different seeds). Random selection results in approximate numbers for total number of rows and actual sizes of each class; deviations from the specified targets are negligible.
- Save the resulting DataFrame in a Parquet file so it can be easily read in and used in multiple downstream notebooks (using Spark). Selecting on the list of identifiers is done by inner-joining it with the full dataset.

[Top **⤒**](#Top)

<a id="Balance"></a>
<div class="alert alert-block alert-info">
<strong>A Note about Churn Rate and Class Balance:</strong>
In a churn use case, time frame is of key importance. For example, in the original use case the data was provided in chunks of one month, one file per month. Each file may have only, say, 1% churners, i.e., customers who were recorded to have left that month. But over the ten months, the overall population size (number of unique customer IDs) grows only incrementally as new customers join up, while the number of churners adds a whole month's worth each month: each month's churners is a unique set, whose customer IDs do not appear the following month.
<br/><br/>
So with an initial population size $N_0$, if the average number of new joiners each month is $\epsilon$ and the average number of churners is $\delta$, with <br/>
$\quad\quad \delta,\epsilon \ll N_0 \quad,$ <br/>
then the population size by the $m^{th}$ month is <br/>
$\quad\quad N_m = N_0 + m(\epsilon - \delta) \quad,$ <br/>
but the <strong>cumulative</strong> number of customer IDs in the dataset (which does not subtract the churners) is <br/>
$\quad\quad \hat{N}_m = N_0 + m\epsilon \quad,$ <br/>
and the cumulative number of churners is <br/>
$\quad\quad \Delta_m = m\delta \quad.$ <br/>
Then the proportion of churners over $m$ months is (using a Taylor expansion)<br/>
$\quad\quad \frac{\Delta_m}{\hat{N}_m} = \frac{m\delta}{N_0 + m\epsilon} = \frac{m\delta}{N_0}\Big(\frac{1}{1+\frac{m\epsilon}{N_0}}\Big)
= \frac{m\delta}{N_0}\big(1 - \frac{m\epsilon}{N_0} + (\frac{m\epsilon}{N_0})^2 - \ldots\big) \approx \frac{m\delta}{N_0} \quad$ if $ m\epsilon \ll N_0.$<br/>
For example, if $\delta = \epsilon = 0.01N_0$ (monthly churn rate and new-customer rate both 1%), then over 10 months the proportion of churners is <br/>
$\quad\quad \frac{10\times0.01N_0}{N_0} \approx 0.1$, or about 10% <br/>
(where the approximation is itself off by less than 10%).
In other words, over a longer time period the class imbalance becomes less severe.
</div>

<a id="Imports"></a>
## Import necessary libraries & packages
And set up the Spark session.

[Top **⤒**](#Top)

In [2]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import *

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import lit
from pyspark.sql.functions import sum
from pyspark.sql.functions import max

spark = SparkSession(sc).builder.getOrCreate()

import os

# Handy packages for reporting running time and wall clock time
from time     import time
from datetime import datetime, timedelta
from pytz     import timezone

import builtins                        # because we need both Spark max() and Python max()

local_path = '/project_data/data_asset'    # Handy for reading and writing to local project

<a id="Parameters"></a>
## Set the parameters for building the ID subset
This consists of the target subset size and the desired fraction of records belonging to the positive class, the names of the ID and label columns, the list of periods index numbers, and the minimum number of periods in which an ID must be present for that ID to be included.

The example code below is tailored to a dataset derived from synthetic [telco churn data](https://dataplatform.cloud.ibm.com/community?context=analytics&query=telco%20&format=dataset) available in the IBM Watson Community. To run this notebook without modification, first run the notebook *Prepare Churn Data* to create that dataset. The sample dataset is so small that the technique demonstrated here is entirely unnecessary. It is used only to demonstrate that the code runs without error.

[Top **⤒**](#Top)

In [3]:
n_sub_ids      = 700           # Set this to your choice of subset size
p_sub_positive = 0.35          # Set this to your desired positive-class fraction (to achieve a good class balance)
id_column      = 'customer_id' # Set this to the name of the unique-identifier column
label_column   = 'churned'     # Set this to the name of the label column
file_sequence  = range(0, 12)  # Set this to the range of periods, one file per period
min_periods    = 3             # Set this to the minimum number of periods of history an ID needs, to be included.
                               # Set to zero of no such criterion applies.

<a id="Timezone"></a>
## Set the browser client timezone
This is only for convenience in reporting progress. When the output says, "Started at <time>" you should be able to see quickly how long it's been running. This is easier when the timestamp is in your own timezone, rather than UTC or the timezone of wherever Watson Studio happens to be installed.

The next cell shows how to get a list of available timezones. To see all of them, remove the slice and ellipsis from the list:
``` python
print('{}'.format('\n'.join(all_timezones)))
```
Or, to see only timezones for a particular region, filter the list:
``` python
print('{}'.format('\n'.join([tz for tz in all_timezones if tz.startswith('US')])))
```

[Top **⤒**](#Top)

In [4]:
# Show which timezones are available.
from pytz import all_timezones

print('{}'.format('\n'.join(all_timezones[:5] + ['...'])))

Africa/Abidjan
Africa/Accra
Africa/Addis_Ababa
Africa/Algiers
Africa/Asmara
...


In [5]:
# Set to local timezone of browser (not the Jupyter client or Watson Studio server or UTC)
# Examples: US/Central, Europe/Berlin, America/Bogota, Asia/Singapore, etc.
tz = timezone('US/Pacific')

<a id="Filenames"></a>
## Define a function for generating filenames
If there are multiple files in a series, named according to some convention, generate filenames in sequence according to that convention. For example, if each file represents one month's worth of data, files could be named `datafile_jan.csv`, `datafile_feb.csv`, etc., or `userdata_01.csv`, `userdata_02.csv`, etc. There is probably a constant base name ("datafile" or "userdata", for example) and some prefix or suffix that distinguishes the months. Any scheme will do, even if you just have to list all filenames explicitly and pick from the list by index.

In addition, you have to specify where the files live (the path).

The sample code follows this scheme: `customers_<MONTH>.csv` (uncompressed CSV files), where `<MONTH>` is `Jan`, `Feb`, ..., `Dec`, in the local project path `/project_data/data_asset`.

[Top **⤒**](#Top)

In [6]:
from calendar import month_abbr   # Used in this sample to generate short month names

file_path = local_path                                      # Set the path to the files, if in this project
file_base = 'customers_'                                    # Set the constant part of the filenames
suffixes  = month_abbr                                      # Use a lookup table for filename suffixes (NOTE: 1=January, 2=February, etc.)
file_type = '.csv'                                          # Set the file type suffix (.csv, .csv.gz, .zip, ...)
 
def full_filename(index, fpath=file_path, fbase=file_base, suff=suffixes, ftype=file_type):
    '''
    Generate a full-path filename for a file in a series, given a (library or project) path, a filename base (the constant part),
    a list of suffixes, from which a numeric index will select, and a file-type suffix (e.g., '.csv.gz').
    
    NOTE: The suffix lookup table in this example is base-1 because it's for months (where 1, not 0, is January, etc.).
          But the index parameter to this function is assumed to be base-0, so we use index + 1.
    '''
    return os.path.join(fpath, fbase + suff[index + 1] + ftype)

<a id="Schema"></a>
## Provide the input schema if necessary
If you maintain the input schema elsewhere and the files are without header, set the schema here.

The schema should be of the form:
``` python
StructType([StructField('COLUMN_ONE', StringType(),  False),
            StructField('COLUMN_TWO', IntegerType(), True),
            ...
            StructField('COLUMN_LAST', StringType(), True)])
```
**NOTE** The third argument in each `StructField()` call indicates whether the column may contain nulls; generally, data scientists are not too fussy about that in DataFrames (people managing databases do tend to be), but it makes sense to assume (and enforce) that the ID and label columns should not contain nulls.

If the input data files do contain a header row, set the schema to `None`. Code in a later cell will tell Spark to infer the schema from the data.

**NOTE** If you ask Spark to infer the schema, it has to make an extra pass over the data, significantly increasing the time it takes to read each file. So
for speed and efficiency, it is actually better to have CSV files without header and maintain their schemas separately. Trade off this consideration against
the convenience of having the files be self-contained.

[Top **⤒**](#Top)

In [8]:
input_schema = None          

<a id="Merge"></a>
## Read and merge the dataset
This is the heart of the notebook, where all the work happens of reading and merging the input data and boiling it down to just the unique IDs and associated labels.

Remember that Spark employs lazy evaluation: transformations like selection, unioning, and aggregation are not actually performed until an action, such as `count()`, forces it to instantiate the result. Normally it makes sense to wait till the very end and let Spark design the most efficient execution plan, knowing all the steps from beginning to end. The problem is that, while Spark is executing, it is difficult to get a sense of progress and an idea of when the operation will complete. Therefore, in the case of reading and merging multiple large files, where each file may take a long time (say, more than 15 minutes), it is helpful to get a periodic report that one file was finished and the next file started. This is why the code reports a row count after every file, even though it is not required for the production of the final result.

Also notice the use of explicit caching (`cache()`). Spark maintains an execution plan for each sequence of transformations, and in the absence of a cache directive it will re-execute that plan each time an action requires instantiation of the result. In this case, that would mean that for each file in the sequence it would read all previous files and regenerate each intermediate result. Despite I/O buffering this adds an enormous amount of unnecessary work and time. If reading each file takes 20 minutes, for example, the first file would take 20 minutes, the second 40 minutes, the third 60 minutes, and so on. If we did not ask for a row count after each file, this would not occur, as Spark would simply accumulate a query plan and execute it only once, when the final merged DataFrame is finally used in an action. Even then, caching of the merged DataFrame would be important, since we'll be asking for a total count before performing the random sampling and writing the result to file.

[Top **⤒**](#Top)

In [14]:
# Start with an empty dataframe with the right schema
# Note that neither of the two columns allows nulls
merge_schema = StructType([StructField(id_column,      StringType(),  False),     # Assume ID column is of type string; adjust if necessary
                           StructField(label_column,   IntegerType(), False),     # Assume label column is numeric in {0,1}
                           StructField('period_count', IntegerType(), False)])    # Number of periods in which this ID appears
df_merged = spark.createDataFrame([], merge_schema)

# Set CSV options depending on whether the schema is already known
if input_schema is None:
    csv_header = True 
    csv_infer  = True
else:
    csv_header = False  # Or True if there is a header line and you just want to skip it
    csv_infer  = False

# Figure out the maximum filename length (for readable output)
# Figure out the maximum number of digits needed for the number of rows later
max_len = len(builtins.max([full_filename(n, fpath='') for n in file_sequence], key=len))
max_rows_len = None

lastrows = 0
print('Started table merge at {}'.format(datetime.now(tz).strftime('%X %x')))  # Use current locale format, if set (it may not be)

# Read in all files
for sequence_number in file_sequence:
    start     = time()
    filename  = full_filename(sequence_number)
    
    df_merged = (spark.read.csv(filename, schema=input_schema,
                                header=csv_header,
                                inferSchema=csv_infer)              # May need to add arguments like sep=, depending on file details
                 .select(id_column, label_column)                   # Drop all columns we don't need
                 .withColumn(label_column,                          # -----------------
                             col(label_column)                      # This is only needed if labels come in as strings, e.g., 'Y' and 'N'
                             .cast("boolean").cast("integer"))      # ----------------- ... turn into 1 and 0, via True and False
                 .withColumn('period_count', lit(1))                # Each period contributes one count
                 .union(df_merged)                                  # Merge with prior (aggregated) results
                 .groupBy(id_column)                                # Aggregate the combination of new data and prior results,
                 .agg(max(label_column  ).alias(label_column  ),    # keeping the label to 0 or 1
                      sum('period_count').alias('period_count'))    # and adding up the number of occurrences
                 .cache()                                           # Because it's used twice: to get the count,
                                                                    # and to union with the next file
                )
    
    # Use an action to instantiate the intermediate-result DataFrame. This may or may not be efficient,
    # but at least you can follow progress.
    rows      = df_merged.count()
    
    # Assume that the first file gives you a lot of rows and the rest just
    # add to that incrementally. Allow for the increments to bring the total
    # up one order of magnitude.
    if max_rows_len is None: max_rows_len = len(str(rows))
    
    elapsed = timedelta(seconds=int(time() - start))
    print('Finished with file {file:{fwid}} in {td}. Current size: {rows:>{rwid1}} rows ({diff:>{rwid}} added).'
          .format(file=os.path.basename(filename), fwid=max_len, td=elapsed, rows=rows,
                  diff=rows-lastrows, rwid1=max_rows_len+1, rwid=max_rows_len))
    lastrows = rows

df_merged.printSchema()

print('{} unique IDs'.format(df_merged.count()))
print('Finished table merge at {}'.format(datetime.now(tz).strftime('%X %x')))

Started table merge at 18:28:52 06/18/20
Finished with file customers_Jan.csv in 0:00:02. Current size:  691 rows (691 added).
Finished with file customers_Feb.csv in 0:00:04. Current size:  718 rows ( 27 added).
Finished with file customers_Mar.csv in 0:00:02. Current size:  749 rows ( 31 added).
Finished with file customers_Apr.csv in 0:00:03. Current size:  779 rows ( 30 added).
Finished with file customers_May.csv in 0:00:03. Current size:  794 rows ( 15 added).
Finished with file customers_Jun.csv in 0:00:03. Current size:  823 rows ( 29 added).
Finished with file customers_Jul.csv in 0:00:03. Current size:  847 rows ( 24 added).
Finished with file customers_Aug.csv in 0:00:03. Current size:  880 rows ( 33 added).
Finished with file customers_Sep.csv in 0:00:03. Current size:  905 rows ( 25 added).
Finished with file customers_Oct.csv in 0:00:03. Current size:  941 rows ( 36 added).
Finished with file customers_Nov.csv in 0:00:03. Current size:  965 rows ( 24 added).
Finished with

### Compute the required fractions and report results
Specify the fractions of the positive and negative classes to be used for sampling as a fraction of the actual size of each subpopulation in the full dataset.

For example, to extract 100k IDs with 25% positives (i.e., 25k IDs with a label of 1) if the full dataset has 1M rows with 10% positives (i.e., 100k 1-valued labels),
specify a fraction of $25k/100k = \frac{1}{4}$ from the positive class. And 75k/900k = $\frac{1}{12}$ from the negative class.

With a total of $N$ IDs in the dataset that satisfy the minimum-number-of-periods criterion, of which $N_+$ are positives, to get a subset of $N_s$ IDs with a desired proportion $p_+$ of positives, the required sampling fraction for the positive class is
$$f_+ = N_s\frac{p_+}{N_+}\quad,$$
and
$$f_- = N_s\frac{1 - p_+}{N - N_+}$$
for the negative class.

In the next cell, $N$ is `n_total`, $N_+$ is `n_positive`, $f_+$ is `f_positive`, and $f_-$ is `f_negative`.

[Top **⤒**](#Top)

In [15]:
# First discard the IDs that have too little history
df_merged = df_merged.filter('period_count >= {}'.format(min_periods))

In [16]:
n_total    = df_merged.count()
n_positive = df_merged.filter(label_column + ' > 0').count()
f_positive = n_sub_ids*p_sub_positive/n_positive                # Obviously, this raises an exception if n_positive = 0;
                                                                # in that case you have a bigger problem anyway
f_negative = n_sub_ids*(1-p_sub_positive)/(n_total-n_positive)

In [17]:
n_len = len(str(n_total))
print('Total unique qualifying IDs: {:>{}}'.format(n_total, n_len))
print('of which are positives:      {:>{}}'.format(n_positive, n_len))
print('Positive-class sampling rate to get {}% positives: {:.3f}'.format(p_sub_positive*100, f_positive))
print('Negative-class sampling rate to get {}% negatives: {:.3f}'.format((1 - p_sub_positive)*100, f_negative))
print('... out of a subset of {} IDs.'.format(n_sub_ids))

Total unique qualifying IDs: 871
of which are positives:      253
Positive-class sampling rate to get 35.0% positives: 0.968
Negative-class sampling rate to get 65.0% negatives: 0.736
... out of a subset of 700 IDs.


<a id="Sample"></a>
## Sample the full list of IDs for the desired total number and class balance
Take advantage of the available Spark method `sampleBy()`, which lets you specify a different fraction
for each of the possible values of a given column (any values not specified are not sampled).

This method extracts a random sample of the specified size. Use a seed for the randomization to get
repeatable results. Vary the seed to create different, independent subsets.

Use multiple extracted subsets to test whether the model is robust with respect to the training set and whether the training set is large enough.
If the model performance varies greatly from one subset to the next, the model is probably overfitting and may need a larger training/validation set.


Note that, due to the randomized approach, the resulting dataset size and class counts may deviate slightly from the specified numbers.

[Top **⤒**](#Top)

In [18]:
df_final = df_merged.sampleBy(col=label_column, fractions={0:f_negative, 1:f_positive}, seed=123)
print('Actual subset size: {} rows'.format(df_final.count()))
df_final.groupBy(label_column).count().show()

Actual subset size: 695 rows
+-------+-----+
|churned|count|
+-------+-----+
|      1|  246|
|      0|  449|
+-------+-----+



<a id="Write"></a>
## Write the selected IDs to disk
Include the class label column.

Write a Parquet file for easy reading into a Spark DataFrame, preserving the schema (column names and types).

Note that for the sample dataset the size inicator in the file name becomes "0k" because we're extracting fewer than 1,000 IDs. In any normal situation, the extracted set should be in the tens or hundreds of thousands.

[Top **⤒**](#Top)

In [19]:
# Name the file for the parameters used: total number of IDs (in thousands) and percentage of positives.
# Adjust path and filename as needed.
full_path = os.path.join(file_path, file_base + '{}k_{}'.format(n_sub_ids//1000, int(p_sub_positive*100)) + '.parquet')
try:
    df_final.write.parquet(full_path, mode='overwrite')
except:
    raise
else:
    print('Parquet file {} successfully written.'.format(full_path))

Parquet file /project_data/data_asset/customers_0k_35.parquet successfully written.


### Developed by IBM Data Science Elite Team, IBM Data Science and AI:
- Robert Uleman, Data Science Engineer

Copyright (c) 2019 IBM Corporation